In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import squidpy as sq
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
import tabulate
import itertools
from mpl_toolkits import mplot3d
import matplotlib.patches as mpatches
import umap
from importlib import reload
from scipy import stats
import matplotlib.image as mpimg
import gget

# locals
import utils as ut
reload(ut)

sc.settings.verbosity = 3

2022-12-20 10:18:05.866688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 10:18:05.976344: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2022-12-20 10:18:05.976360: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-20 10:18:06.002196: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-20 10:18:06.8684

In [2]:
clusterPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/scanpy/clusters.csv"
cf = pd.read_parquet(clusterPath)
print(cf.shape)
print(cf['cellType'].unique())
cf.head()

(427882, 7)
['Macrophages' 'B cells' 'Monocytes' 'NK cells' 'Dendritic cells'
 'T cells' 'Stromal cells']


,gene,scores,pvals,logfoldchanges,clusterId,key,cellType
0,H2-EB1,36.343353,4.575346e-181,3.564091,1,ND,Macrophages
1,H2-AA,35.311523,4.761517e-183,3.430534,1,ND,Macrophages
2,H2-AB1,34.411163,2.815112e-177,3.234597,1,ND,Macrophages
3,CD74,33.910587,2.451588e-177,3.302062,1,ND,Macrophages
4,H2-DMA,31.695400,2.991793e-123,2.667091,1,ND,Macrophages


In [3]:
dirPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/SPT/"

min_counts = 10

adata = {}

for f in os.listdir(dirPath):
    
    if not f == "all":
        key = f
        mtxPath = f"{dirPath}{f}/outs/"
        data = sc.read_visium(mtxPath)
        data.var_names_make_unique()
        data.var_names = [x.upper() for x in data.var_names]
        
        """ filter the SPT data"""
        # sc.pp.filter_cells(data, min_counts=min_counts)
        # sc.pp.normalize_total(data, target_sum=1e4) # Normalize each cell by total counts over all genes
        # sc.pp.log1p(data) # Logarithmize data via `X = \log(X + 1)`,
        # sc.pp.scale(data) # unit variance and zero mean
        
        adata[key] = data
    
adata.keys()

reading /nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/SPT/HFD14/outs/filtered_feature_bc_matrix.h5
 (0:00:00)
reading /nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/SPT/ND/outs/filtered_feature_bc_matrix.h5
 (0:00:00)
reading /nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/SPT/HFD8/outs/filtered_feature_bc_matrix.h5
 (0:00:00)


dict_keys(['HFD14', 'ND', 'HFD8'])

In [4]:
dirPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/scanpy/"

rna = {}

for f in os.listdir(dirPath):
    
    if f.endswith('.h5ad'):
        key = f.split(".")[0]
        fullPath = f"{dirPath}{f}"
        data = sc.read(fullPath)
        rna[key] = data
    
print("done")

done


In [5]:
dirPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/scRNAseq/"

raw = {}

for f in os.listdir(dirPath):
    
    if not f == "all":
        key = f
        mtxPath = f"{dirPath}{f}"
        data = sc.read_10x_mtx(mtxPath, cache=True)
        data.var_names = [x.upper() for x in data.var_names] # uppercase gene names
        raw[key] = data
    
print("done")

... reading from cache file cache/nfs-turbo-umms-indikar-shared-projects-spatial_transcriptomics-data-scRNAseq-HFD14-matrix.h5ad
... reading from cache file cache/nfs-turbo-umms-indikar-shared-projects-spatial_transcriptomics-data-scRNAseq-ND-matrix.h5ad
... reading from cache file cache/nfs-turbo-umms-indikar-shared-projects-spatial_transcriptomics-data-scRNAseq-HFD8-matrix.h5ad
done


In [6]:
# load cluster assignments
cdf = pd.read_csv("clusterAssignments.csv")
cid = {}

for key in cdf['key'].unique():
    t = cdf[cdf['key'] == key]
    cid[key] = t
    
print('done')

done


In [7]:
def getImage(data):
    """A function to get image from the spatial data """
    from scanpy.pl._tools import scatterplots as spt    
    sptData = data.uns['spatial']
    
    libraryId = list(sptData.keys())[0]
    sptData = sptData[libraryId]
    
    # get params
    img = sptData['images']['hires']
    scale_factor = sptData['scalefactors']['tissue_hires_scalef']
    return img, scale_factor

In [9]:
keys = ['ND', 'HFD8', 'HFD14']

outdir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/cleanData/"

clusters = []
rnas = []
spts = []

for key in keys:
    spt = adata[key]
    scr = rna[key]
    rawRna = raw[key]
    
    _, sf = getImage(spt)
    
    # get coordinates
    coords = pd.DataFrame(spt.obsm['spatial'])
    coords.columns = ['x', 'y']
    coords['x'] = coords['x'] * sf
    coords['y'] = coords['y'] * sf
    
    # get the run key
    allKeys = list(scr.obs.keys() )
    allKeys.remove('n_genes')
    runKey = allKeys[0]
    clusterIds = scr.obs[runKey].astype(int) + 1  # note that these are zero-indexed!
    clusterIds = pd.DataFrame(clusterIds)
    clusterIds = clusterIds.reset_index(drop=False)
    clusterIds.columns = ['cellId', 'clusterId']
    
    clusterAssignments = cid[key][['clusterId', 'cellType']]
    
    clusterIds = pd.merge(clusterIds, 
                          clusterAssignments,
                          how='left',
                          left_on='clusterId',
                          right_on='clusterId')
    
    src = rawRna.to_df()
    spt = spt.to_df()
    
    coords.index = spt.index
    
    print(key, src.shape, spt.shape, coords.shape)
    
    outpath = f"{outdir}{key}_clusters.pq"
    clusterIds.to_parquet(outpath, index=False)
    
    outpath = f"{outdir}{key}_coords.pq"
    coords.to_parquet(outpath, index=True)
    
    outpath = f"{outdir}{key}_rna.pq"
    src.to_parquet(outpath, index=True)
    
    outpath = f"{outdir}{key}_spt.pq"
    spt.to_parquet(outpath, index=True)
    
print('done')

ND (1414, 31053) (2036, 31053) (2036, 2)
done ND
HFD8 (6558, 31053) (3394, 31053) (3394, 2)
done HFD8
HFD14 (7146, 31053) (1994, 31053) (1994, 2)
done HFD14
